# <center>Разведочный анализ данных (EDA). Часть 1</center>

Данный ноутбук построен на основе гайда от разработчиков `pandas`, который можно найти по ссылке https://pandas.pydata.org/docs/user_guide/10min.html \
Сразу предупрежу что по этой ссылке можно застрять надолго. Поэтому советую сначала прочитать ноутбук ниже и ознакомиться с ссылками из него, а потом перейти к гайду от разработчиков и ознакомиться с ним.

Для начала импортируем библиотеки, которые нам понадобятся в этом ноутбуке.
Советую все импорты выносить в начало ноутбука, а не раскидывать по разным частям =)

In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
pd.__version__

'1.2.4'

## Установка Anaconda
1. Перейти на сайт Anaconda: https://www.anaconda.com/download/
2. Выбрать операционную систему, которая стоит у вас на компьютере (Windows, Linux или MacOS)
3. Скачать соответствующий разрядности Вашего процессора (32 бит или 64 бит) вариант Anaconda
4. Для установки библиотеки jupyter: conda install jupyter
5. Запустить ноутбук: jupyter notebook

#### Список базовых команд для работы с anaconda:
- Создать новое окружение:
`conda create --name test_env python=3.6`
- Посмотреть уже созданные окружения:
`conda info --envs`
- Активировать окружение:
`conda activate test_env`
- Выйти из окружения в изначальное (базовое):
`conda activate`
- Установить новый пакет:
`conda install beautifulsoup4`
- Посмотреть все установленные пакеты:
`conda list`
- Обновить пакет:
`conda update beautifulsoup4`

## Работа с Series

`pd.Series` - это индексированный массив данных. \
У этой структуры есть индекс элементов, который может являться любой хешируемый объект и элементы, которыми может являться любой объект (даже другой `pd.Series`). \
`pd.Series` очень похож на словарь из стандартных структур Python, но в дополнение к этому обладает множеством методов для обработки. Большинство этих методов написаны так, что вам будет проще работать с `pd.Series`, если вы будете его воспринимать как массив элементов с индексом.

Для начала определим простой `pd.Series` с названием apples, у которого индексом будут строки, а элементами будут целые числа. На его примере разберемся с простыми методами, которые доступны нам "из коробки".

In [80]:
apples = pd.Series([10, 30, 20, 25, 4, 0], 
                   index = ['Alise', 'Andrew', 'Bob', 'Matt', 'Charles', 'Ann']) 
apples

Alise      10
Andrew     30
Bob        20
Matt       25
Charles     4
Ann         0
dtype: int64

Для `pd.Series` доступны методы получение максимального, минимального, среднего, медианного и тп элементов.

In [4]:
apples.max(), apples.min(), apples.mean(), apples.median()

(30, 0, 14.833333333333334, 15.0)

Индексирование элементов доступно по индексам этих элементов. Если индекс - это строка, то есть 2 вида обращения к элементу.

In [5]:
apples['Andrew'], apples.Andrew

(30, 30)

Также обращаться можно по "нумерному" индексу, как в стандартном списке.

In [6]:
apples[1]

30

Также для `pd.Series` есть понятие среза, которые работают аналогично срезам в стандартном списке.

In [84]:
apples[:]

Alise      10
Andrew     30
Bob        20
Matt       25
Charles     4
Ann         0
dtype: int64

Можно пользоваться фильтрацией элементов через удобный `numpy` синтаксис.\
Фактически алгоритм работы такой:
1. Строим булеву маску (apples > 15) элементов `pd.Series`, которая будет содержать `True` на месте элемента, для которого выполняется условие и `False` на месте элементов, для которых не выполняется условие.
2. Индексируемся по булевой маске, оставляя только те элементы `pd.Series`, для которых булева маска содержит `True` на месте элемента.

In [8]:
mask = apples > 15
mask

Alise      False
Andrew      True
Bob         True
Matt        True
Charles    False
Ann        False
dtype: bool

In [88]:
apples[mask]

Andrew    30
Bob       20
Matt      25
dtype: int64

In [10]:
apples[apples > 15]

Andrew    30
Bob       20
Matt      25
dtype: int64

Фильтрацию можно делать на основе нескольких булевых масок и объединяются они не логическими, а битовыми операциями (например, битовое И, которое в Python обозначается в виде `&`)

In [94]:
apples[(apples > 15) & (apples < 30)]

Bob     20
Matt    25
dtype: int64

Бекенд библиотеки `pandas` был построен с использованием библиотеки `numpy`, про которую мы вам расскажем в следующий раз. И во многом `pandas` полагается и многое отнаследовал от библиотеки `numpy`. В том числе свой "математический" `np.nan`, в отличие от "объектного" `None` из Python. \
Спустя несколько лет и версию `pandas` 1.0 появился новый None, но теперь свой встроенный (`pd.NA`).

In [12]:
pd.NA, None, np.nan

(<NA>, None, nan)

Давайте разберемся как работать с неполными данными и как нам может помочь `pandas`.

In [131]:
apples['Carl'] = pd.NA
apples

Alise        10
Andrew       30
Bob          20
Matt         25
Charles       4
Ann           0
Carl       <NA>
dtype: object

Если мы понимаем "природу данных" или как они были получены, то можем заполнить эти значения (с помощью метода `pd.fillna`).\
Например, мы можем заполнить пропущенные данные числом, которое передали сами, или числом, которое было вычисленно на остальных данных (например, медиана).

In [127]:
apples.fillna(-999)

Alise       10
Andrew      30
Bob         20
Matt        25
Charles      4
Ann          0
Carl      -999
dtype: int64

In [132]:
apples.fillna(apples.median())

Alise      10.0
Andrew     30.0
Bob        20.0
Matt       25.0
Charles     4.0
Ann         0.0
Carl       15.0
dtype: float64

Можно заметить, что при заполнении пропущенного значения медианой наш `pd.Series` поменял свой тип с `int64` на `float64`. А если посмотреть чуть выше, то можно заметить, что `pd.Series`, содержащий `pd.NA`, был типа `object`.\
Это связано с тем, что `pd.Series` умеет сам определять тип данный и менять тип элементов, если этого требует его модификация.\
Мы можем поменять тип сами, если это возможно. Для этого надо использовать метод `pd.astype`.

In [133]:
apples.fillna(apples.median().astype(int))

Alise      10
Andrew     30
Bob        20
Matt       25
Charles     4
Ann         0
Carl       15
dtype: int64

Теперь давайте познакомимся с аргументом `inplace`, который часто встречается в методах `pandas`. Этот аргумент "меняет" принцип работы метода.
- Если `inplace=False` (по умолчанию), то метод возвращает модифицированный `pd.Series`.
- Если `inplace=True`, то метод изменяет переданный `pd.Series` и возвращает `None`.

In [17]:
apples.fillna(apples.median(), inplace=True)
apples

Alise      10.0
Andrew     30.0
Bob        20.0
Matt       25.0
Charles     4.0
Ann         0.0
Carl       15.0
dtype: float64

Небольшая помарка относительно типов данных. В `pandas` их достаточно много и это может сильно путать в самом начале.\
Простой пример ниже, мы используем тип `Int64`, который отличается от типа, использованного выше `int64`.\
Более подробно про типы можно прочитать в документации, если не боитесь английского и много букв: https://pandas.pydata.org/pandas-docs/stable/user_guide/basics.html#basics-dtypes

In [18]:
apples.astype('Int64')

Alise      10
Andrew     30
Bob        20
Matt       25
Charles     4
Ann         0
Carl       15
dtype: Int64

Размер `pd.Series` можно узнать стандартной для Python функции `len` или с помощью метода `pd.shape`. В первом случае вернется целое число, во втором tuple из 1 элемента.

In [19]:
len(apples[apples > 15])

3

In [139]:
apples.shape

7

## Работа с DataFrame

`pd.DataFrame` - основной тип данных, который нам предоставляет библиотека `pandas`.\
`pd.DataFrame` - это таблица, например, как в Excel, или как набор `pd.Series` со сквозным индексов для элементов и своим именем (название столбца).\
Бекенд `pd.DataFrame` представляет из себя набор `pd.Series` с парой индексов.

Сначала разберемся с тем как можно создавать `pd.DataFrame`. Для этого есть 2 основных подхода:
1. Отдельно указывать матрицу с данными и 2 индекса строк и колонок.

In [186]:
df1 = pd.DataFrame(np.random.randn(5, 3), 
                   index=['o1', 'o2', 'o3', 'o4', 'o5'], 
                   columns=['f1', 'f2', 'f3'])
df1

,f1,f2,f3
o1,-2.185236,-0.549955,-0.849877
o2,-0.375494,0.094652,0.710512
o3,-0.040600,0.198146,-0.836793
o4,-0.053404,-0.602549,-1.212883
o5,-1.339325,1.363126,-1.482896


2. Задавать `pd.DataFrame` в виде словаря. То есть каждую колонку оформлять в виде элемента словаря, а ключ - это название индекса колонки.

In [187]:
df2 = pd.DataFrame({'A': np.random.random(5), 
                    'B': ['a', 'b', 'c', 'd', 'e'], 
                    'C': np.arange(5) > 2})
df2

,A,B,C
0,0.139940,a,False
1,0.423570,b,False
2,0.819828,c,False
3,0.782679,d,True
4,0.670449,e,True


Обращаться к элементам в `pd.DataFrame` можно разнообразными способами, каждый из которых имеет свою плюсы и минусы.\
Основные проблемы связаны с получением "ссылки на элемент" (View) или копии элемента (Copy). В документации есть большой документ проясняющий принцип работы: http://pandas-docs.github.io/pandas-docs-travis/user_guide/indexing.html
1. Выбирать колонку или элемент с помощью оператора квадратных скобок

In [154]:
df1['f1']['o2']

-0.2496569010143646

In [156]:
df1['f3']

o1    1.006918
o2    0.484246
o3    0.006049
o4    3.322487
o5    1.160741
Name: f3, dtype: float64

В этом случае вы делаете 2 последовательных вызова (Chained Indexing), сначала выбраете колонку (`pd.Series`), а потом в нем обращаетесь к элементу.\
В этом случае вы не знаете что получили по итогу: копию элемента или "ссылку на элемент".\
Это не проблема при просмотре значений, но может стать проблемой при присваивании.

2. Выбирать элемент с помощью оператора `pd.at`

In [157]:
df1.at['o3', 'f1']

-1.1698654702337619

Можно выбрать только один элемент, но всегда возвращает "ссылку на элемент".

3. Выбирать элемент с помощью оператора `pd.iat`

In [160]:
df1.iat[2, 0]

-1.1698654702337619

Работает аналогично `pd.at`, но подаются 2 целочисленных индекса, когда в `pd.at` должны подаваться имена индексов.

4. Выбирать подмножество элементов с помощью операторов `pd.loc` или `pd.iloc`.\
Разница в операторах аналогично разнице `pd.at` / `pd.iat`.\
В отличие от `pd.at` оператор `pd.loc` позволяет выбрать подмножество элементов (датафрейм на основе старого, колонку или элемент).\
Если вы передаете в `pd.loc` не список элементов для одной из осей, то всегда возвращаются "ссылки на элементы".

In [27]:
df2.loc[3, 'B']

'd'

In [163]:
df2.loc[1:3, ['B', 'C']]

,B,C
1,b,False
2,c,False
3,d,True


Теперь перейдем к некоторым примерам как можно изменять `pd.DataFrame`. Некоторые из них всегда работают, а некоторые нет в зависимости от правил, описанных выше.

In [29]:
df2.at[2, 'B'] = 'F'
df2

,A,B,C
0,0.567910,a,False
1,0.518998,b,False
2,0.209409,F,False
3,0.814364,d,True
4,0.739334,e,True


В данном примере хочется отметить, что мы слева от присваивания стоит множетсво из 2 элементов, а справа мы передаем только 1 элемент. Но все работает.\
Это связано с умением `pandas` уравнивать оси, если это необходимо. Более подробно мы об этом поговорим на следующей занятии по `numpy` и тут это работает полностью аналогично.

In [168]:
df1.loc[3:5, 'f1'] = 'FFF'
df1

/Users/an.loskutov/Documents/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  """Entry point for launching an IPython kernel.


,f1,f2,f3,B
o1,QQQ,-0.107039,1.006918,QQQ
o2,QQQ,-0.604688,0.484246,QQQ
o3,QQQ,1.553039,0.006049,QQQ
o4,FFF,-0.201414,3.322487,QQQ
o5,FFF,-1.148296,1.160741,QQQ


То же самое `pandas` умеет делать и для пары `pd.Series`. Также тут можно заметить, что `pandas` сам подставляет необходимые типы и тут мы без ошибок можем их смешивать или заменять.

In [177]:
df2[['B', 'C']] = 4
df2

,A,B,C
0,0.667063,4,4
1,0.403631,4,4
2,0.686152,4,4
3,0.474066,4,4
4,0.145718,4,4


In [172]:
df2.loc[df2.A > 0.3, 'B'] = [1, 2, 3, 4]
df2

,A,B,C
0,0.667063,1,False
1,0.403631,2,False
2,0.686152,3,False
3,0.474066,4,True
4,0.145718,e,True


In [181]:
df2.loc[:, 'B'] = 'F'
df2.loc[5, :] = 3.1415
df2

,AAAAA,B,C
0,0.667063,F,4.0000
1,0.403631,F,4.0000
2,0.686152,F,4.0000
3,0.474066,F,4.0000
4,0.145718,F,4.0000
5,3.141500,3.1415,3.1415


In [34]:
df2.iloc[2, 0] = 14.31
df2

,A,B,C
0,0.567910,3.0000,4.0000
1,0.518998,3.0000,4.0000
2,14.310000,4.0000,4.0000
3,0.814364,3.0000,4.0000
4,0.739334,3.0000,4.0000
5,3.141500,3.1415,3.1415


Теперь перейдем к тому как можно менять название индексов у строк или колонок. Для этого есть метод `pd.rename`, который я и советую использовать.

In [35]:
row_mapping = {x: f"column{idx}" for idx, x in enumerate(df1.index, 1)}
print(f"Row mapping: {row_mapping}")
df1.rename(row_mapping)

Row mapping: {'o1': 'column1', 'o2': 'column2', 'o3': 'column3', 'o4': 'column4', 'o5': 'column5'}


,f1,f2,f3
column1,-1.046866,0.687201,-0.728417
column2,-1.143862,0.494658,0.856795
column3,-0.022745,-0.275290,-0.135069
column4,-0.141896,-1.738494,-0.064064
column5,0.211241,-1.505141,-0.527437


Для того, чтобы контролировать что вы переименовываете (индекс колонок или строк) надо изменять аргумент `axis`.

In [36]:
df2 = df2.rename({'B': 'BBBBB', 'C': 'CCCCC'}, axis=1)
df2

,A,BBBBB,CCCCC
0,0.567910,3.0000,4.0000
1,0.518998,3.0000,4.0000
2,14.310000,4.0000,4.0000
3,0.814364,3.0000,4.0000
4,0.739334,3.0000,4.0000
5,3.141500,3.1415,3.1415


Если еще один способ изменить название колонок, можно это сделать напрямую поменял поле в `pd.DataFrame`, которое за это отвечает.

In [180]:
df2.columns = ['AAAAA', 'B', 'C']
df2

,AAAAA,B,C
0,0.667063,4.0000,4.0000
1,0.403631,4.0000,4.0000
2,0.686152,4.0000,4.0000
3,0.474066,4.0000,4.0000
4,0.145718,4.0000,4.0000
5,3.141500,3.1415,3.1415


Ниже на примерах разберем несколько полезных методов для работы с `pd.DataFrame`

In [188]:
df1.columns = ['A', 'B', 'C']
df3 = df1.append(df2, sort=False)
df3

,A,B,C
o1,-2.185236,-0.549955,-0.849877
o2,-0.375494,0.094652,0.710512
o3,-0.040600,0.198146,-0.836793
o4,-0.053404,-0.602549,-1.212883
o5,-1.339325,1.363126,-1.482896
0,0.139940,a,0.000000
1,0.423570,b,0.000000
2,0.819828,c,0.000000
3,0.782679,d,1.000000
4,0.670449,e,1.000000


In [189]:
df1.at['o2', 'A'] = np.nan
df1.at['o4', 'C'] = np.nan
df1

,A,B,C
o1,-2.185236,-0.549955,-0.849877
o2,NaN,0.094652,0.710512
o3,-0.040600,0.198146,-0.836793
o4,-0.053404,-0.602549,NaN
o5,-1.339325,1.363126,-1.482896


In [40]:
pd.isnull(df1)

,A,B,C
o1,False,False,False
o2,True,False,False
o3,False,False,False
o4,False,False,True
o5,False,False,False


In [191]:
df1.dropna(how='any')

,A,B,C
o1,-2.185236,-0.549955,-0.849877
o3,-0.040600,0.198146,-0.836793
o5,-1.339325,1.363126,-1.482896


In [196]:
df1.fillna(df1.median())

,A,B,C
o1,-2.185236,-0.549955,-0.849877
o2,-0.696365,0.094652,0.710512
o3,-0.040600,0.198146,-0.836793
o4,-0.053404,-0.602549,-0.843335
o5,-1.339325,1.363126,-1.482896


## Примеры базового анализа

В данном блоке попробуем загрузить данные и потом их немного проанализировать.\
Как можно заметить, если обратиться к документации (https://pandas.pydata.org/docs/user_guide/io.html) `pandas` поддерживает достаточно много вариантов "считать откуда-то данные".\
На примере снизу представлен пример "скраппинга сайта" или считывание данных из файла, если их уже кто-то достал.

In [200]:
filename = 'nba.csv'

if not os.path.exists(filename):
    tables = pd.read_html("http://www.basketball-reference.com/leagues/NBA_2016_games.html")
    games = tables[0]
    games.to_csv(filename)
else:
    games = pd.read_csv(filename)
games.head()

,Unnamed: 0,Date,Start (ET),Visitor/Neutral,PTS,Home/Neutral,PTS.1,Unnamed: 6,Unnamed: 7,Attend.,Notes
0,0,"Tue, Oct 27, 2015",8:00p,Cleveland Cavaliers,95,Chicago Bulls,97,Box Score,NaN,21957,NaN
1,1,"Tue, Oct 27, 2015",8:00p,Detroit Pistons,106,Atlanta Hawks,94,Box Score,NaN,19187,NaN
2,2,"Tue, Oct 27, 2015",10:30p,New Orleans Pelicans,95,Golden State Warriors,111,Box Score,NaN,19596,NaN
3,3,"Wed, Oct 28, 2015",7:00p,Washington Wizards,88,Orlando Magic,87,Box Score,NaN,18846,NaN
4,4,"Wed, Oct 28, 2015",7:30p,Philadelphia 76ers,95,Boston Celtics,112,Box Score,NaN,18624,NaN


Для начала переименуем часть колонок, чтобы было проще работать с `pd.DataFrame`.

In [201]:
column_names = {'Date': 'date', 'Start (ET)': 'start',
                'Unamed: 2': 'box', 'Visitor/Neutral': 'away_team', 
                'PTS': 'away_points', 'Home/Neutral': 'home_team',
                'PTS.1': 'home_points', 'Unamed: 7': 'n_ot'}

games = games.rename(columns=column_names)
games.head()

,Unnamed: 0,date,start,away_team,away_points,home_team,home_points,Unnamed: 6,Unnamed: 7,Attend.,Notes
0,0,"Tue, Oct 27, 2015",8:00p,Cleveland Cavaliers,95,Chicago Bulls,97,Box Score,NaN,21957,NaN
1,1,"Tue, Oct 27, 2015",8:00p,Detroit Pistons,106,Atlanta Hawks,94,Box Score,NaN,19187,NaN
2,2,"Tue, Oct 27, 2015",10:30p,New Orleans Pelicans,95,Golden State Warriors,111,Box Score,NaN,19596,NaN
3,3,"Wed, Oct 28, 2015",7:00p,Washington Wizards,88,Orlando Magic,87,Box Score,NaN,18846,NaN
4,4,"Wed, Oct 28, 2015",7:30p,Philadelphia 76ers,95,Boston Celtics,112,Box Score,NaN,18624,NaN


Теперь оставим только некоторые колонки, с которыми будем работать, и выкинем те строки, где число известных значений не менее 4 (аргумент `thresh` у метода `pd.dropna`).

In [202]:
games = games.dropna(thresh=4)[['date', 'away_team', 'away_points', 'home_team', 'home_points', 'Attend.']]
games.head(9)

,date,away_team,away_points,home_team,home_points,Attend.
0,"Tue, Oct 27, 2015",Cleveland Cavaliers,95,Chicago Bulls,97,21957
1,"Tue, Oct 27, 2015",Detroit Pistons,106,Atlanta Hawks,94,19187
2,"Tue, Oct 27, 2015",New Orleans Pelicans,95,Golden State Warriors,111,19596
3,"Wed, Oct 28, 2015",Washington Wizards,88,Orlando Magic,87,18846
4,"Wed, Oct 28, 2015",Philadelphia 76ers,95,Boston Celtics,112,18624
5,"Wed, Oct 28, 2015",Chicago Bulls,115,Brooklyn Nets,100,17732
6,"Wed, Oct 28, 2015",Utah Jazz,87,Detroit Pistons,92,18434
7,"Wed, Oct 28, 2015",Indiana Pacers,99,Toronto Raptors,106,19800
8,"Wed, Oct 28, 2015",Charlotte Hornets,94,Miami Heat,104,19724


`pd.DataFrame` умеет работать не только со стандартными типами данных, но и с datetime, например.\
Для того чтобы воспользоваться этим функционалом приведем колонку `date` к нужному типу.\
На данном курсы мы затронем даннный тип данных немного поверхностно, но если вы много работаете со временем у себя в данных, то советую ознакомиться с возможностями `pandas` по ссылке https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries

In [205]:
games.date = pd.to_datetime(games['date'], format='%a, %b %d, %Y')
games.head(3)

,date,away_team,away_points,home_team,home_points,Attend.
0,2015-10-27,Cleveland Cavaliers,95,Chicago Bulls,97,21957
1,2015-10-27,Detroit Pistons,106,Atlanta Hawks,94,19187
2,2015-10-27,New Orleans Pelicans,95,Golden State Warriors,111,19596


`pd.DataFrame` предоставляет прекрасные 2 метода для просмотра части данных: `pd.head` и `pd.tail`, которые отображают соответственно первые 5 строк или последние 5 строк. Число строк можно отдельно указать в параметрах.\
С помощью данных методов можно удобно смотреть на часть своих данных, проверяя проделанные манипуляции.

In [206]:
games.tail(3)

,date,away_team,away_points,home_team,home_points,Attend.
35,2015-10-31,Brooklyn Nets,91,Memphis Grizzlies,101,16013
36,2015-10-31,Phoenix Suns,101,Portland Trail Blazers,90,17906
37,2015-10-31,Sacramento Kings,109,Los Angeles Clippers,114,19060


Перед тем как перейти к анализу давайте посмотрим на то, как изучать размерность `pd.DataFrame`\
Для этого как и в `pd.Series` есть функция `len`, которая возвращает число строк и метод `pd.shape`, который возвращает кортеж из 2 элементов: число строк и число столбцов.

In [208]:
print(games.shape)
print(len(games))

(38, 6)
38


С помощью метода `pd.dtypes` можно посмотреть на типы колонок. Это бывает необходимо, когда вы пытаетесь понять что у вас за данных (если вы их только получили) или, например, для отладки проблем.

In [49]:
games.dtypes

date           datetime64[ns]
away_team              object
away_points             int64
home_team              object
home_points             int64
Attend.                 int64
dtype: object

С помощью метода `pd.describe` можно составить небольшой анализ данных, которые у нас есть.

In [50]:
games.describe()

,away_points,home_points,Attend.
count,38.000000,38.000000,38.000000
mean,103.657895,101.263158,18274.763158
std,13.187374,13.951042,1547.135163
min,75.000000,71.000000,13858.000000
25%,94.000000,94.250000,17678.000000
50%,103.000000,102.000000,18323.000000
75%,112.000000,110.750000,19155.250000
max,139.000000,136.000000,21957.000000


In [51]:
games.describe(include=['object', 'datetime64[ns]'])

/Users/an.loskutov/Documents/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


,date,away_team,home_team
count,38,38,38
unique,5,26,26
top,2015-10-28 00:00:00,Utah Jazz,Phoenix Suns
freq,14,3,2
first,2015-10-27 00:00:00,NaN,NaN
last,2015-10-31 00:00:00,NaN,NaN


In [52]:
games.describe(percentiles=[0.1, 0.9, 0.9995])

,away_points,home_points,Attend.
count,38.000000,38.000000,38.000000
mean,103.657895,101.263158,18274.763158
std,13.187374,13.951042,1547.135163
min,75.000000,71.000000,13858.000000
10%,90.100000,82.900000,16639.100000
50%,103.000000,102.000000,18323.000000
90%,117.300000,113.300000,19803.600000
99.95%,138.907500,135.926000,21931.192500
max,139.000000,136.000000,21957.000000


Также есть метод для сортировки значений `pd.sort_values`

In [53]:
games.sort_values(by='away_points', ascending=False).head()

,date,away_team,away_points,home_team,home_points,Attend.
22,2015-10-30,Oklahoma City Thunder,139,Orlando Magic,136,18846
34,2015-10-31,Golden State Warriors,134,New Orleans Pelicans,120,18406
9,2015-10-28,New York Knicks,122,Milwaukee Bucks,97,18717
26,2015-10-30,Washington Wizards,118,Milwaukee Bucks,113,13858
33,2015-10-31,New York Knicks,117,Washington Wizards,110,20356


In [212]:
games.sort_values(by=['away_points', 'home_points'], ascending=[True, False]).head()

,date,away_team,away_points,home_team,home_points,Attend.
27,2015-10-30,Brooklyn Nets,75,San Antonio Spurs,102,18418
6,2015-10-28,Utah Jazz,87,Detroit Pistons,92,18434
19,2015-10-29,Dallas Mavericks,88,Los Angeles Clippers,104,19218
3,2015-10-28,Washington Wizards,88,Orlando Magic,87,18846
35,2015-10-31,Brooklyn Nets,91,Memphis Grizzlies,101,16013


Теперь перейдем к некоторым методам аналитике данных. То есть попробуем использовать знания, которые были описаны выше и, комбинируя их, ответить на следующие вопросы:

**Какой средний балл у команд, которые играли дома?**

In [55]:
games['home_points'].mean()

101.26315789473684

**Какой максимальный балл у команд, которые играли на выезде 28 октября 2015?**

In [56]:
games[games['date'] == '2015-10-28'].head()

,date,away_team,away_points,home_team,home_points,Attend.
3,2015-10-28,Washington Wizards,88,Orlando Magic,87,18846
4,2015-10-28,Philadelphia 76ers,95,Boston Celtics,112,18624
5,2015-10-28,Chicago Bulls,115,Brooklyn Nets,100,17732
6,2015-10-28,Utah Jazz,87,Detroit Pistons,92,18434
7,2015-10-28,Indiana Pacers,99,Toronto Raptors,106,19800


In [57]:
games.loc[games['date'] == '2015-10-28', 'away_points'].max()

122

**Какой минимальный балл у команд, которые играли дома, 30 октября 2015 года, и на матчах которых было более 18500 зрителей?**

In [58]:
games.loc[(games['date'] == '2015-10-30') & (games['Attend.'] > 18500), 'home_points'].min()

102

### Теперь перейдем к более сложным способам анализа, в которых `pandas` и его функциональность проявляется в полном объеме.

Для начала немного о методе `pd.apply`. Этот метод применяет переданную в него функцию для выбранной оси `pd.DataFrame`.\
Чтобы применить функцию к каждому элементу `pd.DataFrame` надо использовать метод `pd.applymap`.\
Для `pd.Series` есть аналогичный `pd.apply` метод `pd.map`.

In [59]:
games[['away_points', 'home_points']].head(10)

,away_points,home_points
0,95,97
1,106,94
2,95,111
3,88,87
4,95,112
5,115,100
6,87,92
7,99,106
8,94,104
9,122,97


In [213]:
def f(x):
    return x * x

games[['away_points', 'home_points']].head(10).applymap(lambda x: x * x)

,away_points,home_points
0,9025,9409
1,11236,8836
2,9025,12321
3,7744,7569
4,9025,12544
5,13225,10000
6,7569,8464
7,9801,11236
8,8836,10816
9,14884,9409


In [61]:
games[['away_points', 'home_points']].head(10).apply(lambda ser: ser.max() - ser.min())

away_points    35
home_points    25
dtype: int64

In [62]:
games[['away_points', 'home_points']].head(10).apply(lambda row: row.median(), axis=1)

0     96.0
1    100.0
2    103.0
3     87.5
4    103.5
5    107.5
6     89.5
7    102.5
8     99.0
9    109.5
dtype: float64

С строками в `pd.Series` можно работать через модификатор `.str` и применять любую стандартную функцию

In [63]:
col = games['away_team']
col.str.split().head()

0      [Cleveland, Cavaliers]
1          [Detroit, Pistons]
2    [New, Orleans, Pelicans]
3       [Washington, Wizards]
4       [Philadelphia, 76ers]
Name: away_team, dtype: object

Один из главных и самых удобных для анализа методов `pandas` является `pd.groupby`.\
Для тех кто знаком с SQL - метод работает аналогично операции GROUP BY в запросах типа SELECT.\

Иначе говоря, строка кода `df.groupby('column1').agg_func` проделывает примерно такую последовательность операций:\
<code>
for unique_element in df['column1'].unique():
    df_group = df[df.column1 == unique_element]
    df_group.set_index('column1')
    new_row_for_unique_element = df_group.apply(agg_func)
</code>
\
После применения возвращается `pd.DataFrame`, состоящий из набора строк `new_row_for_unique_element` для каждого униклаьного элемента из колонки 'column1'.

In [216]:
games.groupby('date').size()

date
2015-10-27     3
2015-10-28    14
2015-10-29     3
2015-10-30    12
2015-10-31     6
dtype: int64

In [226]:
games.groupby('date').mean()

,away_points,home_points,Attend.
date,,,
2015-10-27,98.666667,100.666667,20246.666667
2015-10-28,103.214286,99.500000,18595.857143
2015-10-29,104.000000,102.666667,19065.000000
2015-10-30,103.083333,102.833333,17500.916667
2015-10-31,108.166667,101.833333,17692.166667


In [230]:
games.groupby('date').get_group('2015-10-31')

,date,away_team,away_points,home_team,home_points,Attend.
32,2015-10-31,Utah Jazz,97,Indiana Pacers,76,14412
33,2015-10-31,New York Knicks,117,Washington Wizards,110,20356
34,2015-10-31,Golden State Warriors,134,New Orleans Pelicans,120,18406
35,2015-10-31,Brooklyn Nets,91,Memphis Grizzlies,101,16013
36,2015-10-31,Phoenix Suns,101,Portland Trail Blazers,90,17906
37,2015-10-31,Sacramento Kings,109,Los Angeles Clippers,114,19060


In [237]:
games.groupby('date').mean()

,away_points,home_points,Attend.
date,,,
2015-10-27,98.666667,100.666667,20246.666667
2015-10-28,103.214286,99.500000,18595.857143
2015-10-29,104.000000,102.666667,19065.000000
2015-10-30,103.083333,102.833333,17500.916667
2015-10-31,108.166667,101.833333,17692.166667


In [234]:
games.groupby('date').agg({'home_points': 'mean',
                           'home_team': 'nunique'})

,home_points,home_team
date,,
2015-10-27,100.666667,3
2015-10-28,99.500000,14
2015-10-29,102.666667,3
2015-10-30,102.833333,12
2015-10-31,101.833333,6


In [238]:
games.groupby('date', as_index=False).agg({'home_points': lambda x: x.mean(),
                                           'away_points': 'mean',
                                           'home_team': 'nunique',
                                           'away_team': 'nunique'})

,date,home_points,away_points,home_team,away_team
0,2015-10-27,100.666667,98.666667,3,3
1,2015-10-28,99.500000,103.214286,14,14
2,2015-10-29,102.666667,104.000000,3,3
3,2015-10-30,102.833333,103.083333,12,12
4,2015-10-31,101.833333,108.166667,6,6


Важно отметить, что метод `pd.groupby` возвращает не `pd.DataFrame`, а объект типа `pd.core.groupby.generic.DataFrameGroupBy`. Который имеет сильно ограниченный набор методов, но их можно использовать в своих целях, чтобы периодически не писать свои велосипеды =)

In [68]:
grouped_games = games.groupby('date')
for dt, sub_df in grouped_games:
    print(dt)
    display(sub_df.head())
    break

2015-10-27 00:00:00


,date,away_team,away_points,home_team,home_points,Attend.
0,2015-10-27,Cleveland Cavaliers,95,Chicago Bulls,97,21957
1,2015-10-27,Detroit Pistons,106,Atlanta Hawks,94,19187
2,2015-10-27,New Orleans Pelicans,95,Golden State Warriors,111,19596


In [70]:
d1 = grouped_games.get_group('2015-10-29')
d1

,date,away_team,away_points,home_team,home_points,Attend.
17,2015-10-29,Memphis Grizzlies,112,Indiana Pacers,103,18165
18,2015-10-29,Atlanta Hawks,112,New York Knicks,101,19812
19,2015-10-29,Dallas Mavericks,88,Los Angeles Clippers,104,19218


Есть метод `pd.value_counts`, который рассчитывает число элементов для каждого уникального значения из переданной колонки (или набора колонок). То есть это более функциональный и простой по аргументом аналог `df.groupby(col).size()`

In [239]:
games['date'].value_counts(sort=True)

2015-10-28    14
2015-10-30    12
2015-10-31     6
2015-10-27     3
2015-10-29     3
Name: date, dtype: int64

In [72]:
games.groupby('date').size()

date
2015-10-27     3
2015-10-28    14
2015-10-29     3
2015-10-30    12
2015-10-31     6
dtype: int64

Хочется обратить внимание на еще несколько важных моментов. `pd.Series` и `pd.DataFrame` имеют много полезных методов для работы с данными. Быстрее и удобнее использовать их, чем пользоваться самописными аналогами и `pd.apply`.\
Это связано как с корректным написанием этих методов, так и очень медленным по производительности `pd.apply`. Так что если вы не пишите что-то совершенно уникальное, то лучше попробовать использовать встроенный функционал `pd.DataFrame`, чем достаточно быстро использовать `pd.apply`.\
Также много стандартных операторов переопределено для `pd.Series` и их можно сразу использовать и без `pd.apply`.

In [73]:
games[['home_points', 'away_points', 'Attend.']].corr()

,home_points,away_points,Attend.
home_points,1.000000,0.416534,0.191471
away_points,0.416534,1.000000,-0.065669
Attend.,0.191471,-0.065669,1.000000


In [74]:
games[['home_points', 'away_points', 'Attend.']].cov()

,home_points,away_points,Attend.
home_points,194.631579,76.633001,4.132740e+03
away_points,76.633001,173.906828,-1.339813e+03
Attend.,4132.739687,-1339.812945,2.393627e+06


Ну и на последок, для любителей Excel, в `pandas` реализованы методы `pd.crosstab` и `pd.pivot_table` для построения соответсвенно перекрестных и сводных таблиц.

In [75]:
games['away_remainder'] = games['away_points'] % 5
games['home_remainder'] = games['home_points'] % 5
pd.crosstab(games['date'], games['away_remainder'])

away_remainder,0,1,2,3,4
date,,,,,
2015-10-27,2,1,0,0,0
2015-10-28,3,4,3,1,3
2015-10-29,0,0,2,1,0
2015-10-30,2,0,3,2,5
2015-10-31,0,2,2,0,2


In [76]:
games.pivot_table(values='Attend.', index=['away_remainder', 'home_remainder'])

Attend.
away_remainder home_remainder              
0              0               17986.000000
               1               19596.000000
               2               19666.333333
               3               17660.000000
1              0               17980.500000
               1               17066.000000
               2               18203.000000
               4               18322.500000
2              0               19205.500000
               1               17740.333333
               2               18963.750000
               3               18165.000000
3              2               18846.000000
               3               15378.000000
               4               19218.000000
4              0               18406.000000
               1               18589.333333
               2               17936.000000
               3               16035.000000
               4               19392.000000

## Классное задание

Ссылка форму: https://vk.cc/c20UB1

**QR код для формы:**

<img src="qr_c20UB1.png" width="300"/>